# Sphinx API documentation

[Sphinx](https://www.sphinx-doc.org) is a powerful documentation tool, based on [reStructuredText](https://www.sphinx-doc.org/en/3.x/usage/restructuredtext/basics.html).

If you are more comfortable with markdown, you can use [MyST](https://myst-parser.readthedocs.io)

In [1]:
ls

conda/                  from-script-to-project/ requirements.txt
env-1/                  pip-tools/              sphinx-docs/
environments.ipynb      pipfile/                sphinx-docs.ipynb


In [2]:
cd sphinx-docs

/Users/minrk/dev/simula/in3110/site/lectures/12-production/sphinx-docs


In [3]:
ls

docs-done/  myarray.py


We can initialize our docs with the command:

    sphinx-quickstart
    
which will prompt with a few questions to set up the docs directory.

There are two key files created (and several more):

1. `conf.py` - the sphinx configuration file
2. `index.rst` the first page of our documentation. This is the landing page for our docs

Let's start by checking out the docs:

    make html

in the docs directory, then open `_build/index.html`


The first thing we are going to do is add the [autodoc](https://www.sphinx-doc.org/en/3.x/usage/extensions/autodoc.html) extension.
`autodoc` is an extension that defines **directives** (commands) for automatically documenting your code.

```python
extensions = [
    "sphinx.ext.autodoc",
]
```

Our module `myarray` defines an `Array` class and an `asarray` function. We want to docment these.
Since we went through the trouble of adding docstrings, it would be nice to import those into our docs! That's what autodoc is for.

Inspecting our module and class, we can see their docstrings:

In [4]:
import myarray
myarray?

Type:        module
String form: <module 'myarray' from '/Users/minrk/dev/simula/in3110/site/lectures/12-production/sphinx-docs/myarray.py'>
File:        ~/dev/simula/in3110/site/lectures/12-production/sphinx-docs/myarray.py
Docstring:  
Defines an Array class.

Class template for Arrays, IN 3110 assignment 3


In [5]:
myarray.Array.mean?

Signature: myarray.Array.mean(self)
Docstring:
Computes the mean of the array

Only needs to work for numeric data types.

Returns:
    float: The mean of the array values.
File:      ~/dev/simula/in3110/site/lectures/12-production/sphinx-docs/myarray.py
Type:      function


Let's get started by creating a new file int he docs directory, with`docs/api.rst`:

```rst
My Array API
============

.. automodule:: myarray

```


and we can try to build the docs again with `make html`.

We will see two warnings:

> ```
> WARNING: autodoc: failed to import module 'myarray'; the following exception was raised:
> No module named 'myarray'
> ```

and

> ```
> docs/api.rst: WARNING: document isn't included in any toctree
> ```

The toctree (**t**able **o**f **c**ontents tree) warning is because our new `api.rst` document is not included in any `.. toc::` directives.
Sphinx wants to make sure that you can navigate through all the documentation
via table of contents, so it encourages you to include all your documents in


We can add our new document to the top-level table of contents in `index.rst`:

```rst
.. toctree::
   :maxdepth: 2
   :caption: Contents:

   api.rst
```

The `autodoc` warning is because our `myarray.py` is not importable, because sphinx is not run from our module directory.

The default `conf.py` has a snippet of code that we can edit to make our module importable:

```python
import os
import sys
sys.path.insert(0, os.path.abspath(os.pardir))
```

This adds the parent directory (`pardir`) of the docs directory to `sys.path`.
That's where our `myarray.py` is, so it will be importable.


*If we have a 'proper' package (with setup.py or pyproject.toml), this is not necessary, because we can always install our package to make it importable).*

No we can build again with `make html` and open `_build/html/api.html`

Next, we can add our function to the documentation:

```rst
My Array API
============

.. automodule:: myarray

.. autofunction:: asarray
```

and build again with `make html`, reloading [_build/html/api.html](sphinx-docs/docs/_build/html/api.html)

Similarly, we can document our `Array` class:

```rst
My Array API
============

.. automodule:: myarray

.. autofunction:: asarray

.. autoclass:: Array
```

now all our top-level items are documented, but we probably want to see our member methods, not just the constructor.

For that, we add the `:members:` option to autoclass:


```rst
My Array API
============

.. automodule:: myarray

.. autofunction:: asarray

.. autoclass:: Array
    :members:
```




And finally, since we are most interested in "special" methods (`__mul__` and friends), we may want to document the special methods as well, which are not included by default in `:members:`:

```rst
My Array API
============

.. automodule:: myarray

.. autofunction:: asarray

.. autoclass:: Array
    :members:
    :special-members:
```

Now we are done setting up our API docs.
[autodoc](https://www.sphinx-doc.org/en/3.x/usage/extensions/autodoc.html) has lots of options for controlling which members are 

The last step we are going to do is enable another extension called [napoleon](https://www.sphinx-doc.org/en/3.x/usage/extensions/napoleon.html),
which understands special docstring formats, which are widely used.
These give nice highlighting and linking for parameters, return types, etc.

```python
extensions = [
    "sphinx.ext.autodoc",
    "sphinx.ext.napoleon",
]
```

Now rebuild, and see how much nicer the output is!